# MLServer, getting started
https://mlserver.readthedocs.io/en/latest/getting-started/index.html

## Model

In [4]:
import spacy
import wikipediaapi

In [5]:
nlp = spacy.load("en_core_web_lg")

In [6]:
wiki_wiki = wikipediaapi.Wikipedia("MyMovieEval (example@example.com)", "en")

In [7]:
# call Python programming language using wiki_wiki
wiki_wiki.page("Python_(programming_language)")

Python_(programming_language) (id: ??, ns: 0)

In [8]:
barbie = wiki_wiki.page("Barbie_(film)").summary
oppenheimer = wiki_wiki.page("Oppenheimer_(film)").summary
print(barbie)
print(f"\n")
print(oppenheimer)

Barbie is a 2023 fantasy comedy film directed by Greta Gerwig from a screenplay she wrote with Noah Baumbach. Based on the eponymous fashion dolls by Mattel, it is the first live-action Barbie film after numerous computer-animated films and specials. The film stars Margot Robbie as the title character and Ryan Gosling as Ken, and follows the pair on a journey of self-discovery following an existential crisis. The supporting cast includes America Ferrera, Kate McKinnon, Issa Rae, Rhea Perlman, and Will Ferrell.
A live-action Barbie film was announced in September 2009 by Universal Pictures with Laurence Mark producing. Development began in April 2014, when Sony Pictures acquired the film rights. Following multiple writer and director changes and the casting of Amy Schumer and later Anne Hathaway as Barbie, the rights were transferred to Warner Bros. Pictures in October 2018. Robbie was cast in 2019, after Gal Gadot turned down the role due to scheduling conflicts, and Gerwig was announc

In [9]:
doc1 = nlp(barbie)
doc2 = nlp(oppenheimer)

In [10]:
doc1.similarity(doc2)

0.985135192228475

## Microservice for serving the model

In [11]:
from mlserver.codecs import decode_args
from mlserver import MLModel
from typing import List
import numpy as np
import spacy


class MyKulModel(MLModel):
    async def load(self):
        self.model = spacy.load("en_core_web_lg")

    @decode_args
    async def predict(self, docs: List[str]) -> np.ndarray:
        doc1 = self.model(docs[0])
        doc2 = self.model(docs[1])

        return np.array(doc1.similarity(doc2))

## Testing

In [12]:
from mlserver.codecs import StringCodec
import requests

In [15]:
inference_request = {
    "inputs": [
        StringCodec.encode_input(
            name="docs", payload=[barbie, oppenheimer], use_bytes=False
        ).dict()
    ]
}
print(inference_request)

{'inputs': [{'name': 'docs', 'shape': [2, 1], 'datatype': 'BYTES', 'parameters': {'content_type': 'str'}, 'data': ['Barbie is a 2023 fantasy comedy film directed by Greta Gerwig from a screenplay she wrote with Noah Baumbach. Based on the eponymous fashion dolls by Mattel, it is the first live-action Barbie film after numerous computer-animated films and specials. The film stars Margot Robbie as the title character and Ryan Gosling as Ken, and follows the pair on a journey of self-discovery following an existential crisis. The supporting cast includes America Ferrera, Kate McKinnon, Issa Rae, Rhea Perlman, and Will Ferrell.\nA live-action Barbie film was announced in September 2009 by Universal Pictures with Laurence Mark producing. Development began in April 2014, when Sony Pictures acquired the film rights. Following multiple writer and director changes and the casting of Amy Schumer and later Anne Hathaway as Barbie, the rights were transferred to Warner Bros. Pictures in October 20

In [16]:
r = requests.post(
    "http://localhost:8080/v2/models/similarity_model/infer",
    json=inference_request,
)
r.json()

{'model_name': 'similarity_model',
 'id': '213cb91d-4d92-4c1a-ac6f-e27419169dfd',
 'parameters': {},
 'outputs': [{'name': 'output-0',
   'shape': [1],
   'datatype': 'FP64',
   'parameters': {'content_type': 'np'},
   'data': [0.985135192228475]}]}

In [17]:
print(
    f"Our movies are {round(r.json()['outputs'][0]['data'][0] * 100, 4)}% similar!"
)

Our movies are 98.5135% similar!
